---
title: Auxiliary Magic
description: Hierarchical regularization via an auxiliary variable magic trick.
date: 7/7/2022
draft: true
bibliography: references.bib
csl: annals_statistics.csl
format:
  html:
    code-fold: true
---

[Last time](https://jlindbloom.github.io/posts/simple_image_deblurring.html) we looked at an image de-blurring problem, which we solved by finding
$$
x^\star = \text{argmin}_x \,\, \| A x - y \|_2^2 + \mathcal{R}(x)
$${#eq-inv_prob_obj}
where the regularization term was
$$
\mathcal{R}(x) = \gamma \| L x \|_2^2 = \gamma x^T L^T L x.
$$
In this post, our goal is to:
- describe a hierarchical prior and a method that can give us a better image reconstruction,
- walk through a "magic" trick that will speed up our method,
- and look at using [`CuPy`](https://cupy.dev/) to accelerate our reconstruction using a GPU.

# Probabilistic inverse problems

While the problem posed in @eq-inv_prob_obj is completely deterministic, we can actually think of it as haven arisen from a probabilistic model. Suppose that
$$
\begin{align*}
x &\sim \mathcal{N}\left( 0, \left(\gamma L^T L \right)^{-1} \right), \\
y \, | \, x &\sim \mathcal{N}\left( A x, I \right).
\end{align*}
$$
Then our corresponding density functions are
$$
\begin{align*}
\pi(x) &\propto \exp\left\{ - \gamma x^T L^T L  x \right\}, \\
\pi(y \, | \, x) &\propto \exp\left\{ - \| A x - y \|_2^2 \right\},
\end{align*}
$$
and by Bayes' theorem the posterior density for $x \, | \, y$ is given as
$$
\pi(x \, | \, y) \propto \exp\left\{ - \| A x - y \|_2^2 \right\} \times \exp\left\{ - \gamma x^T L^T L  x \right\}.
$$
The [MAP estimate](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation) of $x$ is then given as
$$
x^\star = \text{argmax}_x \,\, \pi(x \, | \, y),
$$
which is equivalent to 
$$
\begin{align*}
x^\star &= \text{argmin}_x \,\, - \log \pi(x \, | \, y) \\
&= \text{argmin}_x \,\, \gamma \| L x \|_2^2 = \gamma x^T L^T L x,
\end{align*}
$$
which is exactly @eq-inv_prob_obj. The role of $\mathcal{R}(x)$ can then be seen as contributing a prior of
$$
\pi(x) \propto \exp\left\{ - \mathcal{R}(x) \right\}
$$
into the inference problem. 

# A hierarchical prior

One reason it can be useful to think probabilistically is because we can motivate different choices of the regularizer $\mathcal{R}(x)$. If we pick
$$
Lx \sim \mathcal{N}\left(0, \frac{1}{\gamma} I \right)
$$
as our prior (which corresponds to $\mathcal{R}(x) = \gamma x^T L^T L x$), then we are saying that we believe \emph{a priori} that the discrete gradients in our image are distributed according to zero-mean Gaussian with variance $\gamma^{-1}$. We can tweak the strength of the prior by adjusting $\gamma$ and in turn its influence on our reconstructed image, but note that the same $\gamma$ governs \emph{all} of the entire discrete gradient in the image. Thus we might think to introduce a hierarchical prior on the discrete gradient that could try to (loosely) capture the fact that in some regions in an image the discrete gradient will be much larger than it is elsewhere. Define the prior
$$
\begin{align*}
\beta^H_{i,j}, \beta^V_{i,j} &\sim \Gamma \left( c, d \right), \\
L x &\sim \mathcal{N} \left(0, B_{\beta} \right),
\end{align*}
$${#eq-hierarchical_prior}
which has density 
$$
\pi(x, \beta) = \pi(x \, | \, \beta) \pi(\beta) \propto \det \left( B_{\beta} \right)^{1/2} \exp\left\{ - x^T L^T B_{\beta} L x  \right\} \times \pi(\beta).
$$
Here $\left( \cdot \right)^{V/H}$ represent the fact that we are assigining two different hyper-parameter to govern the gradient in each the vertical and horizontal directions, $\Gamma\left(c, d \right)$ represents the [gamma density function](https://en.wikipedia.org/wiki/Gamma_distribution), 
$$
B_{\beta} = \text{diag}\left( \beta^V_{1,1}, \ldots, \beta^V_{m,n}, \beta^H_{1,1}, \ldots, \beta^H_{m,n} \right),
$$
and
$$
\pi(\beta) \propto \left( \prod_{i,j}^{mn} \Gamma( \beta_{i,j}^H | c, d) \right) \times \left( \prod_{i,j}^{mn} \Gamma( \beta_{i,j}^V | c, d) \right)
$$
meaning that all hyper-parameters are assumed to be independent of one another. The reason we use a $\Gamma$ distribution for the hyper-parameter is because it is a [conjugate prior](https://en.wikipedia.org/wiki/Conjugate_prior) for a Gaussian, meaning that we can determine certain relevant conditional distributions analytically. 

# The new posterior

Using our prior @eq-hierarchical_prior, the full posterior in our original model (with an extra factor of $\frac{1}{2}$) is given as
$$
\pi(x, \beta \, | \, y) \propto \exp\left\{ - \frac{1}{2} \| A x - y \|_2^2 \right\} \times \det \left( B_{\beta} \right)^{1/2} \exp\left\{ - \frac{1}{2} x^T L^T B_{\beta} L x  \right\} \times \pi(\beta).
$${eq-hierarchical_posterior}
In the sampling setting, a common approach that could be used to draw samples from this posterior is [Gibbs sampling](https://en.wikipedia.org/wiki/Gibbs_sampling), which involves iteratively drawing from the conditionals of each variable in the density given all of the others. In this case, our conditionals would be
$$
\begin{align*}
\pi(x \, | \, \beta, y) &\propto \exp\left\{ - \| A x - y \|_2^2 \right\} \times  \exp\left\{ - x^T L^T B_{\beta} L x  \right\}, \\
\pi(\beta \, | \, x, y) &\propto \det \left( B_{\beta} \right)^{1/2} \exp\left\{ - x^T L^T B_{\beta} L x  \right\} \times \pi(\beta).
\end{align*}
$$
Due to conjugacy, we can recognize the first conditional to be a Gaussian and the second to be a Gamma distribution. Specifically, we have $x \, | \, \beta, y \sim \mathcal{N}\left(\mu_1, Q_1^{-1} \right)$ with
$$
\begin{align*}
Q_1 &= A^T A + L^T B_{\beta} L, \\
\mu_1 &= Q_1^{-1} A^T y.
\end{align*}
$$
For $\beta \, | \, x, y$ we have
$$
\begin{align*}
\beta_{i,j} = \Gamma\left( \frac{1}{2} + c, \frac{1}{2}[L x]_{i,j}^2 + d \right),
\end{align*}
$$
where how you deal with $\beta^V$/$\beta^H$ becomes clear when you think about the shapes of these operations.

Rather than code a Gibbs sampling algorithm, we will consider the \emph{Bayesian coordinate descent} (BCD) of [@Glaubitz2022], which is essentially an optimization technique that iteratively draws from the mean of each the conditionals. We iterate
$$
\begin{align*}
x^{k+1} &= \mathbb{E}_{\pi}\left( x \, | \, \beta^k, y \right), \\
\beta^{k+1} &= \mathbb{E}_{\pi}\left( \beta \, | \, x^{k+1}, y \right),
\end{align*}
$$
until the solution converges. One thing to note is that we will generally not settle into a global minimum, since the posterior (with no restrictions on $c$ and $d$) is not log-concave due to the use of a Gamma hyper-prior on the $\beta_{i,j}$.



# An inconvenience

For the purposes of this post, we will assume that $A$ is a BCCB blurring matrix and that $L$ is a BCCB approximation to the discrete gradient. These were the nice assumptions in the previous post that allowed us to reduce all of our linear system solves to FFTs/IFFs and diagonal matrix operations, as well as efficiently sample in the Fourier domain.  

Note that while we have an analytic expression for the distribution $x \, | \beta, y$, we can no longer use our BCCB assumptions to avoid linear system solves. Letting
$$
\begin{align*}
A &= F^H \Lambda F, \\
L &= F^H \Pi F,
\end{align*}
$$
inserting this into $Q_1^{-1} = \left( A^T A + L^T B_{\beta} L \right)^{-1}$ we now get
$$
\begin{align*}
Q_1^{-1} &= \left( A^T A +  L^T B_{\beta} L \right)^{-1} \\
&= \left( F^H \Lambda F F^H \Lambda F +  F^H \Pi F B_{\beta} F^H \Pi F \right)^{-1} \\
&= F^H \left( \Lambda^2 F +  \Pi F^H B_{\beta} F \Pi \right)^{-1} F \\
&= F^H \left( \Lambda^2 +  F^H B_{\beta} F \right)^{-1} F  \\
\end{align*}
$$
where unlike before we are now stuck. The problem is that $B_{\beta}$ is "sandwiched" in-between $F^H$ and $F$, which prevents us from simplifying with $F^H F = I$. While it is not the end of the world to have to solve a linear system using a sparse solver or some other method, we still really would like to use the BCCB assumption to our advantage if we can.



# An auxiliary variable magic trick

It turns out there is a way to get around this, via a "magic trick" of [@Marnissi2018]. For the moment let us fix $B_{\beta}$ to be a constant matrix. Out of the blue, define a new random variable via
$$
\begin{align*}
    u \, | \, x, y &\sim \mathcal{N}\left( H x, Q^{-1}  \right),
\end{align*}
$$
where we are free to choose $H$ and $Q$ (as long as they define a valid distribution). Then, if we consider the joint density of $u, x \, | \, y$ we have that 
$$
\begin{align*}
\pi(u, x \, | \, y) &= \pi(u \, | \, x, y) \times \pi(x \, | \, y) \times \pi(x) \\
&\propto \exp\left\{ -\frac{1}{2} \left( H x - u \right)^T Q \left( H x - u \right)  \right\} \times \exp\left\{ -\frac{1}{2} x^T L^T B L x  \right\} \times \pi(x \, | \, y) \\
&\propto \exp\left\{ -\frac{1}{2} \left[ \left( H x - u \right)^T Q \left( H x - u \right) + x^T L^T B L x  \right] \right\} \times \pi(x \, | \, y) \\
&\propto \exp\left\{ -\frac{1}{2} \left[ x^T H^T Q H x + u^T Q u - 2 x^T H^T Q u + x^T L^T B L x  \right] \right\} \times \pi(x \, | \, y) \\
&\propto \exp\left\{ -\frac{1}{2} \left[ x^T P x + u^T Q u - 2 x^T H^T Q u  \right] \right\} \times \pi(x \, | \, y) \\
\end{align*}
$$
where we have defined
$$
P = H^T Q H + L^T B L.
$$
We are free to choose $H$ and $Q$, so let's make the magic choice
$$
\begin{align*}
H &= L, \\
Q &= \frac{1}{\lambda} I  - B.
\end{align*}
$$
Here $\lambda > 0$ is a constant we are free to choose, so long as $\lambda < \frac{1}{\| B \|}$ which makes $Q$ positive semi-definite as is required of a valid precision matrix. Inserting this choice *and conditioning on* $u$, we obtain
$$
P = \frac{1}{\lambda} L^T L
$$
and
$$
\begin{align*}
\pi(x \, | \, u,  y) &\propto \exp\left\{ -\frac{1}{2} \left[ \frac{1}{\lambda} x^T L^T L x - 2 x^T L^T Q u  \right] \right\} \times \pi(x \, | \, y). \\
\end{align*}
$$
Now, we define yet another random variable
$$
v = Qu = \left( \frac{1}{\lambda} I  - B \right) u,
$$
insert into the density above to get
$$
\begin{align*}
\pi(x \, | \, v,  y) &\propto \exp\left\{ -\frac{1}{2} \left[ \frac{1}{\lambda} x^T L^T L x - 2 x^T L^T v \right] \right\} \times \pi(x \, | \, y), \\
\end{align*}
$$
and finally complete the square in the exponential to finally arrive at
$$
\begin{align*}
\pi(x \, | \, v,  y) &\propto \exp\left\{ -\frac{1}{2 \lambda} \| L x - \lambda v  \|_2^2  \right\} \times \pi(x \, | \, y), \\
\end{align*}
$$
which is the density of the Gaussian $\mathcal{N}\left( \mu_2, Q_2^{-1} \right)$ where
$$
\begin{align*}
Q_2 &= \frac{1}{\lambda} L^T L + A^T A, \\
\mu_2 &= Q_2^{-1} \left( A^T y +  L^T \left( \frac{1}{\lambda} I \right) \left( \lambda v \right)  \right) \\
&= Q_2^{-1} \left( A^T y +  L^T  v \right).
\end{align*}
$$
Note that we could do this since the missing term in the square was independent of $x$. Now we have what we wanted: *the matrix* $B$ *has disappeared* after conditioning on $v$, and we can now take advantage of our BCCB assumption for this conditional. This is all only useful so long as the conditional distribution $v \, | \, x, y $ is also nice, which is the Gaussian $\mathcal{N}\left( \mu_3, Q_3^{-1} \right)$ where
$$
\begin{align*}
Q_3^{-1} &= B, \\
\mu_3 &= B L x,
\end{align*}
$$
which is nice to work with.




# The augmented hierarchical model

It turns out that when we allow $B = B_{\beta}$ to vary with hyper-parameters, everything we just did stays exactly the same, except we must choose a new $\lambda=  \lambda(B_{\beta})$ such that $\lambda < \frac{1}{\| B_\beta \|}$ for the current values of the hyper-parameters. So applying this auxiliary variable magic trick to @eq-hierarchical_posterior, we now consider the alternative posterior with density
$$
\pi(x, v, \beta \, | \, y) \propto \exp\left\{ - \frac{1}{2} \left( v - B_{\beta} L x \right)^T B_{\beta}^{-1} \left( v - B_{\beta} L x \right) \right\} \times \exp\left\{ - \frac{1}{2} \| A x - y \|_2^2 \right\} \times \det \left( B_{\beta} \right)^{1/2} \exp\left\{ - \frac{1}{2} x^T L^T B_{\beta} L x  \right\} \times \pi(\beta).
$$
We have already discussed the conditionals $x \, | \, v, \beta, y$ and $v \, | \, x, \beta, y$ which are the same as before, but the conditional that changes is $\beta \, | \, x, v, y$ which has density
$$
\pi(\beta \, | \, x, v, y) \propto \exp\left\{ - \frac{1}{2} \left( v - B_{\beta} L x \right)^T B_{\beta}^{-1} \left( v - B_{\beta} L x \right) \right\} \times \det \left( B_{\beta} \right)^{1/2} \exp\left\{ - \frac{1}{2} x^T L^T B_{\beta} L x  \right\} \times \pi(\beta).
$$
However, we can determine this by just updating the gamma hyper-prior twice so that this conditional is